In [27]:
#| default_exp classes.DomoCard

In [28]:
#| exporti
from dataclasses import dataclass, field

import asyncio
from fastcore.basics import patch_to

import domolibrary.routes.card as card_routes

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.utils.DictDot as util_dd

In [29]:
# | export
@dataclass
class DomoCard:
    id: str
    auth: dmda.DomoAuth = field(repr=False)
    title: str = None
    description: str = None
    type: str = None
    urn: str = None
    chart_type: str = None
    dataset_id: str = None

    certification: field(default_factory=dict) = None
    owners: field(default_factory=list) = None

    def __post_init__(self):
        # self.Definition = CardDefinition(self)
        pass

    def display_url(self) -> str:
        return f'https://{self.domo_instance}.domo.com/kpis/details/{self.id}'

    @classmethod
    async def _from_json(cls,
                  card_obj,
                   auth: dmda.DomoAuth):

        import domolibrary.classes.DomoUser as dmu
        import domolibrary.classes.DomoGroup as dmg

        dd = card_obj
        if isinstance(card_obj, dict):
            dd = util_dd.DictDot(card_obj)

        card = cls(
            auth=auth,
            id=dd.id,
            title=dd.title,
            description=dd.description,
            type=dd.type,
            urn=dd.urn,
            certification=dd.certification,
            chart_type=dd.metadata.chartType,
            dataset_id = dd.datasources[0].dataSourceId if dd.datasources else None
        )

        tasks = []
        for user in dd.owners:
            if user.type =='USER':
                tasks.append(dmu.DomoUser.get_by_id(user_id=user.id, auth=auth))
            if user.type == 'GROUP':
                tasks.append(dmg.DomoGroup.get_by_id(group_id=group.get('id'), auth=auth))

        card.owners = await asyncio.gather( *tasks)

        return card


In [30]:
@patch_to(DomoCard, cls_method=True)
async def get_by_id(cls: DomoCard, card_id: str,
                      auth: dmda.DomoAuth, debug_api: bool = False):

    res = await card_routes.get_card_metadata(auth=auth,
                                              card_id=card_id, debug_api=debug_api)

    if not res.is_success:
        raise Exception('unable to retrieve card {card_id}')

    domo_card = await cls._from_json(res.response, auth)

    return domo_card


In [31]:
import os
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoCard.get_by_id(card_id=1766265020, auth=token_auth)

DomoCard(id=1766265020, title='Vendor Amounts Dates', description='', type='kpi', urn='1766265020', chart_type='badge_vert_stackedbar', dataset_id='fd70d83b-38dc-40a2-9349-357ec4321d3e', certification=DictDot(state='NOT_CERTIFIED', adminCertified=False), owners=[DomoUser(id='1893952720', title=None, department='Business Improvement', display_name='Jae Wilson1', email_address='jae@onyxreporting.com', role_id=1, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)])

In [32]:
#| hide
import nbdev
nbdev.nbdev_export()